<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Template para o Colab do Projeto Semestral**
---

Atenção, podem ser que nem todas as tarefas sejam executadas no Colab (a aplicação por exemplo, pode estar hospedada no streamlit cloud). Mas a maior parte pode estar aqui ou ao menos indicada e comentada.


Além disso a entrega deve incluir:

1. **Um GitHub público do projeto**
2. **Código completo e executável em um notebook Python (este template)**
3. **Uma aplicação streamlit para consumo do modelo**
4. **Um texto/artigo do projeto**
5. **Um vídeo (link YouTube ou outro) de no máximo 3min de apresentação do projeto**

Um **`readme.md`** no GitHub público do projeto deve indicar (um índice) cada uma dessas entregas.








In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10402554, Gabriella Braz' #@param {type:"string"}
Aluno2 = '10402264, Giovana Liao' #@param {type:"string"}
Aluno3 = '10400630, Maria Julia de Pádua' #@param {type:"string"}


In [12]:
#@title Assinale aqui a sua opção de Projeto
Projeto = "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos" #@param ["IA Aplicada a Imagens: Uso de Modelos de Redes Neurais", "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos"]




# **Resumo**

**1. Objetivo do projeto**

O projeto tem como objetivo construir um modelo de aprendizado de máquina capaz de classificar livros em gêneros literários a partir de suas descrições textuais. A classificação automática de gêneros visa auxiliar plataformas literárias, sistemas de recomendação e bibliotecas digitais, tornando a organização e o acesso ao conteúdo mais eficientes.

**2. Fontes dos dados e dados originais**

A base de dados original foi obtida a partir de um dataset público disponível no Kaggle, contendo cerca de 10 mil registros de livros com informações como título, descrição e lista de gêneros associados. A partir dessa base bruta, foi criada a BASE_MODELO, com os campos renomeados e ajustados para facilitar o processamento. A lista de gêneros, originalmente armazenada como string de listas, foi convertida para listas reais, permitindo a expansão de múltiplos gêneros por livro em diferentes linhas do dataset.

**3. Ferramentas/pacotes de IA utilizados**

Para o pré-processamento textual, foram utilizados recursos da biblioteca NLTK, como remoção de pontuação, stopwords, lematização e tokenização. A vetorização dos textos foi realizada com TfidfVectorizer do scikit-learn, configurado com n-gramas até 5 palavras e limite de frequências para filtrar ruído textual. O modelo de classificação escolhido foi a Regressão Logística, configurada com class_weight="balanced" para lidar com o desbalanceamento entre gêneros. A base foi dividida estratificadamente em treino e teste, e a performance foi avaliada com acurácia e métricas detalhadas (precision, recall, F1-score).

**4. Prévia dos resultados**

Após diversas iterações e melhorias no pré-processamento, o modelo atingiu uma acurácia de aproximadamente 65,1% ao classificar livros entre os 10 gêneros mais frequentes. O desempenho variou de acordo com a quantidade de exemplos por classe, com maior precisão em categorias amplamente representadas, como Fiction, Fantasy e Nonfiction. A estratégia de expansão dos gêneros por linha e a escolha cuidadosa dos parâmetros de vetorização contribuíram significativamente para a melhoria da performance geral.

# **Apresentação dos dados**

Inclua link, amostras dos dados.

In [2]:
import pandas as pd
import ast

# Carregamento da base de dados
df = pd.read_excel("BASE_MODELO.xlsx")

# Filtrar registros válidos
df = df.loc[df["lista_de_generos"] != "[]"]
df = df.dropna(subset=["descricao"])

# Função para converter string de lista em lista real
def to_list(gen_str):
    try:
        return ast.literal_eval(gen_str)
    except:
        return []

# Converter lista de gêneros e explodir
df["genero_principal"] = df["lista_de_generos"].apply(to_list)
df = df.explode("genero_principal")

# Filtrar os 10 gêneros mais comuns
top_generos = df["genero_principal"].value_counts().head(10).index
df = df[df["genero_principal"].isin(top_generos)]

# Remover duplicatas por título de livro
df = df.drop_duplicates("livro")


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Preparação e transformação dos dados**

A etapa de preparação dos dados foi fundamental para garantir a qualidade do modelo de classificação. Inicialmente, os dados foram importados a partir de um arquivo .xlsx previamente estruturado com base em um dataset original do Kaggle. Este arquivo continha aproximadamente 10 mil registros de livros, com informações como título, descrição e uma lista de gêneros atribuídos a cada obra.

Uma das primeiras ações de limpeza foi eliminar entradas que não possuíam descrição textual e aquelas em que a lista de gêneros estava vazia. Em seguida, a coluna que armazenava os gêneros — originalmente no formato de string representando listas — foi convertida para listas reais utilizando ast.literal_eval(). Essa transformação possibilitou o uso do método explode() do pandas, permitindo que cada gênero de um livro fosse tratado como uma instância separada, ou seja, um mesmo livro poderia aparecer mais de uma vez no conjunto de dados, cada linha associada a um de seus gêneros. Isso garantiu maior granularidade para a tarefa de classificação.

Após essa expansão, os textos das descrições passaram por um rigoroso pré-processamento textual. Essa etapa incluiu a remoção de números, pontuações e palavras irrelevantes (stopwords), além da normalização das palavras por meio da lematização — técnica que reduz palavras às suas formas base. Essa transformação foi realizada utilizando bibliotecas da NLTK, como WordNetLemmatizer e stopwords.

Com os textos processados, foi realizada a vetorização utilizando o método TF-IDF (Term Frequency-Inverse Document Frequency), que transforma os textos em representações numéricas baseadas na frequência e relevância dos termos. O vetor foi configurado para extrair n-gramas de 1 a 5 palavras, com um limite de palavras muito comuns e muito raras para evitar ruído e overfitting.

Por fim, foram selecionados os 10 gêneros mais frequentes no dataset, e o dataframe foi filtrado para manter apenas os registros pertencentes a essas categorias. Essa decisão teve como objetivo equilibrar o modelo e melhorar a capacidade preditiva, evitando o viés causado por classes pouco representadas. Também foi garantido que apenas um registro por livro fosse mantido para evitar duplicidade de informações na base final utilizada para treino e teste.


In [7]:
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Downloads NLTK
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

# Inicializações
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Função de pré-processamento do texto
def preprocess_text(text):
    text = re.sub(r"\d+", "", str(text))
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text.lower())
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return " ".join(tokens)

# Aplicar pré-processamento
df["descricao_processada"] = df["descricao"].apply(preprocess_text)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Fine Tuning do modelo**
Após a construção do pipeline inicial de classificação de gêneros literários com base na descrição dos livros, foi conduzido um processo de fine tuning do modelo com o objetivo de maximizar a acurácia e melhorar o equilíbrio entre precisão e recall para as principais classes. A estratégia de ajuste fino concentrou-se em três frentes principais: seleção de atributos, balanceamento das classes e otimização dos hiperparâmetros do algoritmo.

A primeira etapa consistiu em aprimorar o processo de vetorização textual com o uso do TfidfVectorizer. Foram testadas diferentes configurações de n-gramas (de unigramas até pentagramas) e valores de corte para min_df e max_df, que controlam a inclusão de termos com baixa ou alta frequência no vocabulário. Esse ajuste teve como objetivo encontrar um equilíbrio entre a expressividade dos textos e o ruído causado por termos irrelevantes ou extremamente específicos.

Em seguida, foi observada uma grande variação na distribuição das classes, ou seja, alguns gêneros estavam muito mais representados do que outros. Para lidar com esse desbalanceamento, foi utilizado o parâmetro class_weight='balanced' no algoritmo de regressão logística, o qual ajusta automaticamente os pesos das classes com base na frequência relativa de cada uma. Essa técnica ajudou o modelo a tratar com mais equidade os gêneros menos frequentes no conjunto de dados.

Por fim, foram realizados testes com diferentes hiperparâmetros do modelo LogisticRegression, como o número máximo de iterações (max_iter) e o parâmetro de regularização C, responsável por controlar o overfitting. O valor de C=1.0 apresentou um bom equilíbrio entre complexidade e generalização, e max_iter foi aumentado para 1000 para garantir a convergência do algoritmo.

O resultado desse processo de fine tuning foi um aumento consistente na acurácia do modelo, atingindo cerca de 65,1%, com melhora significativa nos scores de F1 em várias classes. Apesar de ainda haver desafios no desempenho de gêneros com baixa representatividade, o modelo demonstrou maior robustez e capacidade preditiva após os ajustes finos aplicados.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Vetorização TF-IDF
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 5),
    min_df=3,
    max_df=0.9
)

X = vectorizer.fit_transform(df["descricao_processada"])
y = df["genero_principal"]

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# **Avaliação do modelo**

A avaliação do modelo de classificação foi conduzida utilizando um conjunto de teste separado (20% dos dados), garantindo uma análise imparcial da performance preditiva da solução. As métricas utilizadas foram acurácia, precisão, recall e F1-score, fornecendo uma visão abrangente do desempenho, especialmente considerando o desequilíbrio entre os gêneros literários.

O modelo final, baseado em regressão logística com vetorização TF-IDF, atingiu uma acurácia global de aproximadamente 65,1%. Embora a acurácia seja uma métrica relevante, ela pode ser enganosa em cenários com classes desbalanceadas, como é o caso de alguns gêneros menos frequentes. Por isso, foi fundamental observar o comportamento do modelo em métricas como o F1-score, que combina precisão e recall, permitindo avaliar o equilíbrio entre falsas classificações positivas e negativas.

A análise do relatório de classificação revelou que os gêneros mais bem representados, como Fiction, Fantasy e Nonfiction, obtiveram F1-scores mais altos, indicando que o modelo conseguiu aprender padrões consistentes nessas categorias. Por outro lado, gêneros com menos amostras, como Contemporary ou Audiobook, apresentaram desempenho inferior, com métricas próximas de zero, refletindo a dificuldade do modelo em generalizar quando há pouca informação disponível para aprendizado.

Além disso, o uso de class_weight='balanced' na regressão logística contribuiu para mitigar parcialmente os efeitos do desbalanceamento, permitindo que o modelo prestasse mais atenção às classes menos frequentes. No entanto, o fato de algumas classes ainda não terem nenhuma amostra corretamente prevista indica a necessidade de estratégias adicionais, como aumento de dados (data augmentation), reamostragem (oversampling/undersampling), ou mudança para modelos mais sofisticados como redes neurais ou transformers.

Em suma, o modelo apresentou desempenho satisfatório dado o cenário e as limitações dos dados. A acurácia de 65,1% e os F1-scores médios por classe indicam que a abordagem tem potencial prático, especialmente para os gêneros mais frequentes, ao mesmo tempo em que destaca oportunidades de melhoria para uma cobertura mais equilibrada de todos os gêneros literários.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Treinamento do modelo
clf = LogisticRegression(max_iter=1000, class_weight="balanced", C=1.0)
clf.fit(X_train, y_train)

# Avaliação
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Acurácia do modelo: {acc:.4f}")
print("\n📊 Relatório de Classificação:")
print(classification_report(y_test, y_pred))



✅ Acurácia do modelo: 0.6511

📊 Relatório de Classificação:
                    precision    recall  f1-score   support

         Audiobook       0.00      0.00      0.00         1
          Classics       0.50      0.67      0.57       146
      Contemporary       0.11      0.05      0.07        19
           Fantasy       0.71      0.75      0.73       214
           Fiction       0.73      0.50      0.59       520
Historical Fiction       0.43      0.63      0.51        94
           Mystery       0.58      0.79      0.67        96
        Nonfiction       0.84      0.81      0.82       447
           Romance       0.46      0.60      0.52        89
       Young Adult       0.42      0.57      0.49        88

          accuracy                           0.65      1714
         macro avg       0.48      0.54      0.50      1714
      weighted avg       0.67      0.65      0.65      1714



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# **Consumo do modelo**

Após o treinamento e validação, o modelo foi integrado a uma função de inferência chamada classificar_livro, que permite prever o gênero literário predominante de um livro com base apenas em sua descrição textual. Esse consumo do modelo foi estruturado de forma simples, eficiente e acessível, tornando possível sua incorporação em aplicações reais, como sistemas de recomendação, bibliotecas digitais ou plataformas de e-commerce de livros.

O fluxo de uso segue três etapas principais: primeiro, o texto da descrição do livro é submetido ao mesmo processo de pré-processamento aplicado durante o treinamento (remoção de números, pontuações, stopwords e lematização). Em seguida, o texto limpo é vetorizado utilizando o modelo TF-IDF previamente treinado. Por fim, a representação vetorial é passada ao classificador de regressão logística, que retorna a previsão de gênero.

Esse modelo pode é consumido diretamente via interface Streamlit. Nessa interface, o usuário pode escolher um livro e visualizar instantaneamente a classificação do gênero sugerida pelo modelo. Esse consumo em tempo real amplia significativamente as possibilidades de aplicação da solução, tornando-a adequada tanto para fins educacionais quanto comerciais.

A leveza do pipeline — com TF-IDF e regressão logística — também garante rápida resposta e baixa demanda computacional, facilitando o uso do modelo mesmo em ambientes com recursos limitados. Com isso, o modelo se torna não apenas funcional e preciso, mas também acessível para diferentes tipos de usuários e plataformas.

In [11]:
# Função para classificação de nova descrição
def classificar_livro(descricao):
    desc_proc = preprocess_text(descricao)
    desc_vect = vectorizer.transform([desc_proc])
    pred = clf.predict(desc_vect)[0]
    return pred


# **Referências**

WOLF, Thomas et al. Transformers: State-of-the-art Natural Language Processing. In: Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing: System Demonstrations. Online: Association for Computational Linguistics, 2020. Disponível em: https://arxiv.org/abs/1910.03771. Acesso em: 09 abr. 2025.

---

In [ ]:
#@title **Avaliação**
GitHub = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao_Model_Code = 7 #@param {type:"slider", min:0, max:10, step:1}

Aplicacao_Streamlit = 9 #@param {type:"slider", min:0, max:10, step:1}

Texto_Artigo  = 6 #@param {type:"slider", min:0, max:10, step:1}

Video = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**

nota = 2*GitHub + 4*Implementacao_Model_Code + 2*Aplicacao_Streamlit + 1*Texto_Artigo + 1*Video

nota = nota / 10

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.9



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.9
1,1115677,DANIEL HENRIQUE,7.9
